In [ ]:
# imports
import os
import re
from dotenv import load_dotenv
import pickle
from sentence_transformers import SentenceTransformer
from testing import Tester
import groq
from pinecone import Pinecone

c:\Users\Muhammad Nasir\Desktop\AutoDealFinder\autoenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
load_dotenv(override=True)
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN')
os.environ['PINECONE_API_KEY'] = os.getenv('PINECONE_API_KEY')

In [5]:
# Load in the test pickle file
with open('test.pkl', 'rb') as file:
    test = pickle.load(file)

In [6]:
def make_context(similars, prices):
    message = "To provide some context, here are some other items that might be similar to the item you need to estimate.\n\n"
    for similar, price in zip(similars, prices):
        message += f"Potentially related product:\n{similar}\nPrice is ${price:.2f}\n\n"
    return message

In [7]:
def messages_for(item, similars, prices):
    system_message = "You estimate prices of items. Reply only with the price, no explanation"
    user_prompt = make_context(similars, prices)
    user_prompt += "And now the question for you:\n\n"
    user_prompt += item.test_prompt().replace(" to the nearest dollar","").replace("\n\nPrice is $","")
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": "Price is $"}
    ]

In [14]:
# Initialize Pinecone
pinecone_api = os.environ['PINECONE_API_KEY']
Pinecone(api_key=pinecone_api)

pc = Pinecone(api_key=pinecone_api)

# Connect to the existing index
index = pc.Index("products")


In [15]:
def description(item):
    text = item.prompt.replace("How much does this cost to the nearest dollar?\n\n", "")
    return text.split("\n\nPrice is $")[0]

In [16]:
description(test[0])

"OEM AC Compressor w/A/C Repair Kit For Ford F150 F-150 V8 & Lincoln Mark LT 2007 2008 - BuyAutoParts NEW\nAs one of the world's largest automotive parts suppliers, our parts are trusted every day by mechanics and vehicle owners worldwide. This A/C Compressor and Components Kit is manufactured and tested to the strictest OE standards for unparalleled performance. Built for trouble-free ownership and 100% visually inspected and quality tested, this A/C Compressor and Components Kit is backed by our 100% satisfaction guarantee. Guaranteed Exact Fit for easy installation 100% BRAND NEW, premium ISO/TS 16949 quality - tested to meet or exceed OEM specifications Engineered for superior durability, backed by industry-leading unlimited-mileage warranty Included in this K"

In [17]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [18]:
def vector(item):
    return model.encode([description(item)])

In [26]:
def find_similars(item):
    query_result = index.query(
        vector=vector(item).astype(float).tolist(),
        top_k=5,
        include_metadata=True
    )

    # Pinecone returns results in 'matches'
    matches = query_result['matches']
    documents = [match['metadata']['category'] for match in matches]
  # if you stored documents as IDs
    prices = [match['metadata']['price'] for match in matches]

    return documents, prices


In [27]:
print(test[1].prompt)

How much does this cost to the nearest dollar?

Motorcraft YB3125 Fan Clutch
Motorcraft YB3125 Fan Clutch Package Dimensions 25.146 cms (L) x 20.066 cms (W) x 15.494 cms (H) Package Quantity 1 Product Type Auto Part Country Of Origin China Manufacturer Motorcraft, Brand Motorcraft, Model Fan Clutch, Weight 5 pounds, Dimensions 10 x 7.63 x 6.25 inches, Country of Origin China, model number Exterior Painted, Manufacturer Part Rank Automotive Automotive Replacement Engine Fan Clutches 583, Domestic Shipping can be shipped within U.S., International Shipping This item can be shipped to select countries outside of the U.S. Learn More, Available October 10, 2007

Price is $225.00


In [28]:
documents, prices = find_similars(test[1])

In [29]:
print(make_context(documents, prices))

To provide some context, here are some other items that might be similar to the item you need to estimate.

Potentially related product:
Tools_and_Home_Improvement
Price is $73.54

Potentially related product:
Automotive
Price is $102.89

Potentially related product:
Automotive
Price is $186.99

Potentially related product:
Automotive
Price is $33.55

Potentially related product:
Automotive
Price is $210.99




In [30]:
print(messages_for(test[1], documents, prices))

[{'role': 'system', 'content': 'You estimate prices of items. Reply only with the price, no explanation'}, {'role': 'user', 'content': 'To provide some context, here are some other items that might be similar to the item you need to estimate.\n\nPotentially related product:\nTools_and_Home_Improvement\nPrice is $73.54\n\nPotentially related product:\nAutomotive\nPrice is $102.89\n\nPotentially related product:\nAutomotive\nPrice is $186.99\n\nPotentially related product:\nAutomotive\nPrice is $33.55\n\nPotentially related product:\nAutomotive\nPrice is $210.99\n\nAnd now the question for you:\n\nHow much does this cost?\n\nMotorcraft YB3125 Fan Clutch\nMotorcraft YB3125 Fan Clutch Package Dimensions 25.146 cms (L) x 20.066 cms (W) x 15.494 cms (H) Package Quantity 1 Product Type Auto Part Country Of Origin China Manufacturer Motorcraft, Brand Motorcraft, Model Fan Clutch, Weight 5 pounds, Dimensions 10 x 7.63 x 6.25 inches, Country of Origin China, model number Exterior Painted, Manufa

In [31]:
def get_price(s):
    s = s.replace('$','').replace(',','')
    match = re.search(r"[-+]?\d*\.\d+|\d+", s)
    return float(match.group()) if match else 0

In [32]:
get_price("The price for this is $99.99")

99.99

In [36]:
from groq import Groq

client = Groq(api_key=os.getenv('GROQ_API_KEY'))

def groq_rag(item):
    documents, prices = find_similars(item)
    response = client.chat.completions.create(
        model="deepseek-r1-distill-llama-70b",  # or another Groq-supported model like "llama3-70b-8192"
        messages=messages_for(item, documents, prices),
        temperature=0.2,
        max_tokens=5,
        seed=42
    )
    reply = response.choices[0].message.content
    return get_price(reply)


In [37]:
groq_rag(test[1])

73.54

In [38]:
test[1].price

225.11

In [39]:
Tester.test(groq_rag, test)

1: Guess: $336.41 Truth: $374.41 Error: $38.00 SLE: 0.01 Item: OEM AC Compressor w/A/C Repair Kit For F...
2: Guess: $73.54 Truth: $225.11 Error: $151.57 SLE: 1.23 Item: Motorcraft YB3125 Fan Clutch
3: Guess: $45.00 Truth: $61.68 Error: $16.68 SLE: 0.10 Item: Dorman 603-159 Front Washer Fluid Reserv...
4: Guess: $349.99 Truth: $599.99 Error: $250.00 SLE: 0.29 Item: HP Premium 17.3-inch HD Plus Touchscreen...
5: Guess: $15.99 Truth: $16.99 Error: $1.00 SLE: 0.00 Item: 5-Position Super Switch Pickup Selector ...
6: Guess: $13.99 Truth: $31.99 Error: $18.00 SLE: 0.62 Item: Horror Bookmarks, Resin Horror Bookmarks...
7: Guess: $39.99 Truth: $101.79 Error: $61.80 SLE: 0.85 Item: SK6241 - Stinger 4 Gauge 6000 Series Pow...
8: Guess: $299.00 Truth: $289.00 Error: $10.00 SLE: 0.00 Item: Godox ML60Bi LED Light Kit, Handheld LED...
9: Guess: $549.00 Truth: $635.86 Error: $86.86 SLE: 0.02 Item: Randall RG75DG3PLUS G3 Plus 100-Watt Com...
10: Guess: $55.99 Truth: $65.99 Error: $10.00 SLE: 0.03 Ite

KeyboardInterrupt: 

## And now to wrap this in an "Agent" class

In [40]:
from agents.frontier_agent import FrontierAgent

In [41]:
# Let's print the logs so we can see what's going on

import logging
root = logging.getLogger()
root.setLevel(logging.INFO)

In [43]:
agent = FrontierAgent(collection=index)

INFO:root:[Frontier Agent] Initializing Frontier Agent
INFO:root:[Frontier Agent] Frontier Agent is set up with groq's DeepSeek-distilled-with-llama
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
INFO:root:[Frontier Agent] Frontier Agent is ready


In [44]:
agent.price("Quadcast HyperX condenser mic for high quality podcasting")

INFO:root:[Frontier Agent] Frontier Agent is performing a RAG search of the Pinecone datastore to find 5 similar products
Batches: 100%|██████████| 1/1 [00:00<00:00,  2.18it/s]
INFO:root:[Frontier Agent] Frontier Agent has found similar products
INFO:root:[Frontier Agent] Frontier Agent is about to call deepseek-r1-distill-llama-70b with context including 5 similar products
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Frontier Agent] Frontier Agent completed - predicting $139.99


139.99

In [45]:
from agents.specialist_agent import SpecialistAgent

In [46]:
agent2 = SpecialistAgent()

INFO:root:[Specialist Agent] Specialist Agent is initializing - connecting to modal
c:\Users\Muhammad Nasir\Desktop\AutoDealFinder\autoenv\Lib\asyncio\events.py:80: DeprecationError: 2025-01-27: `modal.Cls.lookup` is deprecated and will be removed in a future release. It can be replaced with `modal.Cls.from_name`.

See https://modal.com/docs/guide/modal-1-0-migration for more information.
  self._context.run(self._callback, *self._args)
INFO:root:[Specialist Agent] Specialist Agent is ready


In [47]:
agent2.price("Quadcast HyperX condenser mic for high quality podcasting")

INFO:root:[Specialist Agent] Specialist Agent is calling remote fine-tuned model
INFO:root:[Specialist Agent] Specialist Agent completed - predicting $189.00


189.0